In [30]:
from transformers import AutoModel, AutoTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch.nn as nn
from transformers import PreTrainedModel

In [ ]:
#model_name = "dmis-lab/biobert-base-cased-v1.1"


In [ ]:
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

/home/shiva.heydari/Desktop/cls_mdu/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/shiva.heydari/Desktop/cls_mdu/env/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [ ]:
#save_dir = "../models/biobert"

In [ ]:
#tokenizer.save_pretrained(save_dir)

('../models/biobert/tokenizer_config.json',
 '../models/biobert/special_tokens_map.json',
 '../models/biobert/vocab.txt',
 '../models/biobert/added_tokens.json',
 '../models/biobert/tokenizer.json')

In [ ]:
#model.save_pretrained(save_dir)

In [2]:
model_path = "../models/biobert"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [28]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
encoder = AutoModel.from_pretrained(model_path)

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 1201.09it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: ../models/biobert
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.decoder.bias               | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.decoder.weight             | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [5]:
sample_data = {
  "text": "short merge text ...",
  "modality": "MR",
  "vendor": "GE",
  "series_type": "FLAIR",
  "plane": "AX",
  "acquisition": "2D",
  "body": "brain",
  "contrast": "with"
}

In [6]:
encoded_sample = {
  "text": "short merge text ...",
  "label_modality": 0,
  "label_vendor": 1,
  "label_series_type": 2,
  "label_plane": 0,
  "label_acquisition": 1,
  "label_body": 3,
  "label_contrast": 0,
}

In [7]:
df_sample = pd.DataFrame([encoded_sample])
print(df_sample)

                   text  label_modality  label_vendor  label_series_type  \
0  short merge text ...               0             1                  2   

   label_plane  label_acquisition  label_body  label_contrast  
0            0                  1           3               0  


In [8]:
type(df_sample["text"].tolist())
type(df_sample["text"])

pandas.core.series.Series

In [ ]:
'''dataset class '''
class DicomText(Dataset):
    def __init__(self, df):
        self.texts = df["text"].tolist()
        self.labels = df[[
            "label_modality",
            "label_vendor",
            "label_series_type",
            "label_plane",
            "label_acquisition",
            "label_body",
            "label_contrast",
        ]].to_dict(orient="list")
       

    def __getitem__(self, idx):
        text = self.texts[idx]
        item = {"text": text}
        item["label_modality"]   = self.labels["label_modality"][idx]
        item["label_vendor"]     = self.labels["label_vendor"][idx]
        item["label_series_type"]= self.labels["label_series_type"][idx]
        item["label_plane"]      = self.labels["label_plane"][idx]
        item["label_acquisition"]= self.labels["label_acquisition"][idx]
        item["label_body"]       = self.labels["label_body"][idx]
        item["label_contrast"]   = self.labels["label_contrast"][idx]
        return item

    def __len__(self):
        return len(self.texts)


In [11]:
data_obj = DicomText(df_sample)
for item in data_obj:
    print(item)
print(data_obj[0])

{'text': 'short merge text ...', 'label_modality': 0, 'label_vendor': 1, 'label_series_type': 2, 'label_plane': 0, 'label_acquisition': 1, 'label_body': 3, 'label_contrast': 0}
{'text': 'short merge text ...', 'label_modality': 0, 'label_vendor': 1, 'label_series_type': 2, 'label_plane': 0, 'label_acquisition': 1, 'label_body': 3, 'label_contrast': 0}


In [13]:
class DicomCollator:
    def __init__(self, tokenizer, max_lenght: int):
        self.tokenizer = tokenizer
        self.max_lenght = max_lenght
    

    def __call__(self, batch):
        
        texts = [item["text"] for item in batch]

        enc = self.tokenizer(

            texts, 
            padding="max_length", 
            truncation=True,
            max_lenght=self.max_lenght,
            return_tensor="pt",
        )

        labels_modality     = torch.tensor([item["label_modality"]     for item in batch], dtype=torch.long)
        labels_vendor       = torch.tensor([item["label_vendor"]       for item in batch], dtype=torch.long)
        labels_series_type  = torch.tensor([item["label_series_type"]  for item in batch], dtype=torch.long)
        labels_plane        = torch.tensor([item["label_plane"]        for item in batch], dtype=torch.long)
        labels_acquisition  = torch.tensor([item["label_acquisition"]  for item in batch], dtype=torch.long)
        labels_body         = torch.tensor([item["label_body"]         for item in batch], dtype=torch.long)
        labels_contrast     = torch.tensor([item["label_contrast"]     for item in batch], dtype=torch.long)

        enc["labels_modality"]    = labels_modality
        enc["labels_vendor"]      = labels_vendor
        enc["labels_series_type"] = labels_series_type
        enc["labels_plane"]       = labels_plane
        enc["labels_acquisition"] = labels_acquisition
        enc["labels_body"]        = labels_body
        enc["labels_contrast"]    = labels_contrast

        return enc

In [23]:
max_lenght = 128
collator = DicomCollator(tokenizer, max_lenght)

train_loader = DataLoader(

    data_obj, 
    batch_size=1, 
    shuffle=True,
    collate_fn=collator
)

In [24]:
for batch in train_loader:
    print(batch)

{'input_ids': [[101, 1603, 14713, 3087, 119, 119, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [29]:
hidden_size = encoder.config.hidden_size
print(hidden_size)

768


In [ ]:
class BioBertMultiHead(nn.Module):
    def __init__(self, encoder: PreTrainedModel, num_classes_dict: dict):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(encoder)
        hidden_size = self.encoder.config.hidden_size

        self.modality_head    = nn.Linear(hidden_size, num_classes_dict["modality"])
        self.vendor_head      = nn.Linear(hidden_size, num_classes_dict["vendor"])
        self.series_type_head = nn.Linear(hidden_size, num_classes_dict["series_type"])
        self.plane_head       = nn.Linear(hidden_size, num_classes_dict["plane"])
        self.acq_head         = nn.Linear(hidden_size, num_classes_dict["acquisition"])
        self.body_head        = nn.Linear(hidden_size, num_classes_dict["body"])
        self.contrast_head    = nn.Linear(hidden_size, num_classes_dict["contrast"])

        self.loss_fct = nn.CrossEntropyLoss()

    def forward(
        self,
        input_ids,
        attention_mask=None,
        labels_modality=None,
        labels_vendor=None,
        labels_series_type=None,
        labels_plane=None,
        labels_acquisition=None,
        labels_body=None,
        labels_contrast=None,
    ):
        enc_out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        pooled = enc_out.pooler_output  # (batch, hidden_size)

        logits_modality    = self.modality_head(pooled)
        logits_vendor      = self.vendor_head(pooled)
        logits_series_type = self.series_type_head(pooled)
        logits_plane       = self.plane_head(pooled)
        logits_acquisition = self.acq_head(pooled)
        logits_body        = self.body_head(pooled)
        logits_contrast    = self.contrast_head(pooled)

        loss = None
        if labels_modality is not None:
            loss = 0.0
            loss = loss + self.loss_fct(logits_modality, labels_modality)
            loss = loss + self.loss_fct(logits_vendor, labels_vendor)
            loss = loss + self.loss_fct(logits_series_type, labels_series_type)
            loss = loss + self.loss_fct(logits_plane, labels_plane)
            loss = loss + self.loss_fct(logits_acquisition, labels_acquisition)
            loss = loss + self.loss_fct(logits_body, labels_body)
            loss = loss + self.loss_fct(logits_contrast, labels_contrast)

        return {
            "loss": loss,
            "logits_modality": logits_modality,
            "logits_vendor": logits_vendor,
            "logits_series_type": logits_series_type,
            "logits_plane": logits_plane,
            "logits_acquisition": logits_acquisition,
            "logits_body": logits_body,
            "logits_contrast": logits_contrast,
        }



In [ ]:
#model construction

In [ ]:
#evaluation